In [1]:
from pocketflow import Node, Flow
from langchain_core.tools import tool
from agentic_blocks.utils.tools_utils import print_tool, create_tool_registry, execute_pending_tool_calls
from agentic_blocks import call_llm, Messages

In [2]:
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

tools = [multiply]

tool_registry = create_tool_registry(tools)

#print_tool(multiply)

In [5]:
class LLMNode(Node):
    def prep(self, shared):
        return shared["messages"]

    def exec(self, messages) -> str:
        model = "qwen/qwen3-235b-a22b-2507"
        response = call_llm(model=model, messages=messages, tools=tools)
        messages.add_response_message(response)
        return messages
    
    def post(self, shared,  prep_res, messages):
        if messages.has_pending_tool_calls():
            return "tool_node"
        else:
            return "answer_node"
        
        
class ToolNode(Node):
    
    def prep(self, shared):
        return shared["messages"]

    def exec(self, messages) -> str:
        print(f"Tool node: {messages.get_pending_tool_calls()}")
        tool_responses = execute_pending_tool_calls(messages, tool_registry)
        messages.add_tool_responses(tool_responses)
        return messages
    
    def post(self, shared,  prep_res, messages):
        return "llm_node"
    

    
class AnswerNode(Node):
    
    def prep(self, shared):
        return shared["messages"]

    def exec(self, messages) -> str:
        print(f"Answer: {messages.get_messages()[-1]['content']}")

In [7]:
llm_node = LLMNode()
tool_node = ToolNode()
answer_node = AnswerNode()

llm_node - "tool_node" >> tool_node
tool_node - "llm_node" >> llm_node
llm_node - "answer_node" >> answer_node 

flow = Flow(llm_node)

messages = Messages(user_prompt="Multiply 3 and 4")

shared = {"messages": messages}

flow.run(shared)



Tool node: [{'tool_name': 'multiply', 'arguments': {'x': 3, 'y': 4}, 'tool_call_id': 'call_7ca0acd61d2f48959da954'}]
Answer: The result of multiplying 3 and 4 is 12.0.
